In [1]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

In [2]:
n_steps = 1001
n_runs_A = 20
n_runs_B = 10

# Read parquet files
table_read_A = pq.read_table(f'sa_results_seed_42.parquet')
table_read_B = pq.read_table(f'sa_results_seed_37.parquet')

# Convert to pandas
data_read_A = table_read_A.to_pandas()
data_read_B = table_read_B.to_pandas()
# print(data_read_A['SimId'].max())
# print(data_read_B['SimId'].max())

# Change RunIds
data_read_B['RunId'] = data_read_B['RunId'] + n_runs_A

# Change SimIds
max_sims_A = data_read_A['SimId'].max()
data_read_A['SimId'] = (data_read_A['SimId'] + n_runs_B * np.floor(data_read_A['SimId'] / n_runs_A)).astype(int)
data_read_B['SimId'] = (data_read_B['SimId'] + n_runs_A * (1 + np.floor(data_read_B['SimId'] / n_runs_B))).astype(int)

# Concatenate
data_concat = pd.concat([data_read_A, data_read_B])

# Re-sort
data_concat = data_concat.sort_values(by=['SimId', 'RunId', 'Step'])
data_concat = data_concat.reset_index(drop=True)

In [3]:
data_concat.head((n_runs_A + n_runs_B) * n_steps + 2)

,RunId,Step,harvest_volume,harvest_nbrs,harvest_prob,fert_comp_ratio_exponent,top_n_sites_percent,Trees,Fungi,Living Trees Total Volume,Infected Trees,Mean Substrate,Mean Soil Fertility,Harvested volume,SimId
0,0,0,55.859375,3,0.820312,0.671875,0.648438,100,50,13751.000000,0,1.0100,0.487769,0.000000,0
1,0,1,55.859375,3,0.820312,0.671875,0.648438,80,34,10569.759921,0,0.9500,0.177334,3312.968217,0
2,0,2,55.859375,3,0.820312,0.671875,0.648438,80,45,10649.989006,11,0.9275,0.143895,0.000000,0
3,0,3,55.859375,3,0.820312,0.671875,0.648438,80,27,10720.647941,11,0.9025,0.140196,0.000000,0
4,0,4,55.859375,3,0.820312,0.671875,0.648438,339,21,11048.021320,11,0.8975,0.125871,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30027,29,998,55.859375,3,0.820312,0.671875,0.648438,398,0,2454.664772,0,2.9225,0.000000,0.000000,29
30028,29,999,55.859375,3,0.820312,0.671875,0.648438,397,0,2460.351743,0,2.9250,0.000000,56.536701,29
30029,29,1000,55.859375,3,0.820312,0.671875,0.648438,400,0,2469.393760,0,2.9275,0.000000,57.519508,29
30030,0,0,161.328125,3,0.820312,0.671875,0.648438,100,50,14455.000000,0,1.0125,0.507353,0.000000,30


In [4]:
# Write parquet file
table = pa.Table.from_pandas(data_concat)
pq.write_table(table, f'sa_results_all.parquet')